In [1]:
import torch
import pickle

In [9]:
W1 = torch.load('params/W1.pt')
W2 = torch.load('params/W2.pt')
b1 = torch.load('params/b1.pt')
b2 = torch.load('params/b2.pt')
C = torch.load('params/C.pt')
bngain = torch.load('params/bngain.pt')
bnbias = torch.load('params/bnbias.pt')
bnmean = torch.load('params/bnmean.pt')
bnstd = torch.load('params/bnstd.pt')
chars = pickle.load(open('params/chars.pkl', 'rb'))
char_index = pickle.load(open('params/char_index.pkl', 'rb'))

In [10]:
def next_char(prev_8):
    prev_8 = torch.tensor(prev_8)
    emb = C[prev_8]
    probs = torch.softmax(torch.tanh(bngain*((emb.reshape(1,-1) @ W1 + b1 - bnmean)/bnstd) + bnbias) @ W2 + b2, dim=1)
    return chars[torch.multinomial(probs, 1).item()]
def get_name():
    name = ''
    prev_8 = [0,0,0,0,0,0,0,0]
    next = next_char(prev_8)
    while next != '!':
        name += next
        prev_8 = prev_8[1:] + [char_index[next]]
        next = next_char(prev_8)
    return name

In [11]:
for i in range(50):
    print(get_name())

kanuara
lathava
tamchilei
anasbani
migasaivi
lavantawya
rinia
araktiya
bhnudhanan
abugi
ameartha
kamshathenan
araish
ilokar
purmiataa
navadhamiya
reshadiy
vibal
chamdina
jahourat
bhalaeshan
chendaran
eshnushan
hitapardin
dharla
sajaesan
velshena
rajalvan
sheorra
shanojhar
matamshari
sestna
yagmee
joyavina
adinbh
aaksiya
yamshathra
vinjazhrab
bhavichan
divajaya
dalbaban
sudgob
ubinara
nasathy
dedgaj
mahyamaliki
ariniyan
ruthishwari
yajithman
athiyerani


In [12]:
def get_normalised_negative_log_likelihood(word):
    prev_8 = [0,0,0,0,0,0,0,0]
    nll = 0
    for c in word:
        emb = C[prev_8]
        probs = torch.softmax(torch.tanh(bngain*((emb.reshape(1,-1) @ W1 + b1 - bnmean)/bnstd) + bnbias) @ W2 + b2, dim=1)
        nll += -torch.log(probs[:,char_index[c]])
        prev_8 = prev_8[1:] + [char_index[c]]
    return nll / len(word)

In [13]:
get_normalised_negative_log_likelihood('arjun').item(), get_normalised_negative_log_likelihood('steve').item()

(2.64601469039917, 3.3632915019989014)